In [ ]:
import glob
import pandas as pd
import numpy as np 
from astropy.time import Time
from astropy.table import Table, vstack
from astropy.io import fits
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import time
from scipy.stats import sem
from datetime import datetime, timezone
from python_modules.muonpipe import usefull_func
from matplotlib.patches import Circle
import matplotlib.patches as patches
from matplotlib.patches import RegularPolygon
import numpy as np

def format_axes(ax,option):
    
    if(option=='width'):
        ax.set_xlabel('Muon radius [deg]')
        ax.set_ylabel('Muon ring width [deg]')
        ax.grid(linestyle="dashed")
        ax.set_ylim(0.042,0.14)
        ax.set_xlim(1.0,1.23)
        ax.legend()
    if(option=='size'):
        ax2.grid(linestyle="dashed")
        ax2.legend(ncol=2)
        ax2.set_xlabel('Muon radius [deg]')
        ax2.set_ylabel('Muon Size [pe]')
        ax2.set_ylim(1800,3000)
        ax2.set_xlim(1.0,1.22)

# Define the path to your .dat file
file_path = '/Users/vdk/Downloads/lst-sim-config-sim-telarray-lst-magic-prod2-v1.4/mirror_CTA-N-LST1_v2019-03-31.dat'

# Lists to hold the extracted data from the first two columns
first_column = []
second_column = []

# Open the file and read the data
with open(file_path, 'r') as file:
    for line in file:
        # Split the line into a list of elements based on whitespace
        columns = line.split()
        # Check if the line has enough columns
        if len(columns) >= 2 and line[0] != '#':
            # Append the data from the first two columns to their respective lists
            first_column.append(float(columns[0]))
            second_column.append(float(columns[1]))

# Output the lists
print("First column:", first_column)
print("Second column:", second_column)
start_date_2019 = datetime.strptime("2019-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2019 = datetime.strptime("2019-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
    
start_date_2020 = datetime.strptime("2020-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2020 = datetime.strptime("2020-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2021 = datetime.strptime("2021-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2021 = datetime.strptime("2021-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2022 = datetime.strptime("2022-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2022 = datetime.strptime("2022-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2023 = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2023 = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()


### Choose cuts on the time based on the datacheks

In [ ]:
files = glob.glob('/Users/vdk/muons2024/v0.9-v0.10/20*/DL1_datacheck_*.h5')
files.sort()

# This takes a few minutes... DON'T RUN IT MORE THAN ONCE, it is not needed! (If you change the 
# sky region or the cuts just execute the cells from "Data selection configuration" onwards)
dummy = []
dummy2 = []
dummy3 = []

missing_flatfield_tables = 0
for i, file in enumerate(files):
    if i%10 == 0:
        print(i, '/', len(files), file)
    try:
        dummy.append(pd.read_hdf(file, 'cosmics_intensity_spectrum'))
        dummy2.append(pd.read_hdf(file, 'runsummary'))
        fftable = pd.read_hdf(file, 'flatfield', errors='ignore')
        dummy3.append(fftable)
    except: 
            # some check files have no flatfield table at all (if calibox was off) 
        missing_flatfield_tables += 1

# cosmics intensity spectra table (subrun-wise):
cis = pd.concat(dummy, ignore_index=True)

# flatfield table (subrun-wise):
flatfield = pd.concat(dummy3, ignore_index=True)

# parameters computed run-wise:
runsummary = pd.concat(dummy2, ignore_index=True)



In [ ]:
runsummary.columns

### Process low_nsb csv muon files

In [ ]:
#muon_files = glob.glob('/Users/vdk/muons2024/all_muon_fits/*')
muon_files = glob.glob('/Users/vdk/muons2024/all_muon_fits/low_nsb_filter/*')
print(sorted(muon_files))
#df_files = [pd.read_csv(muon_file, na_values=['NA', '?']) for muon_file in sorted(muon_files)[0:5]] # [0:5] for 19-22 years, [5:] for 22-23 years
df_files = [pd.read_csv(muon_file, na_values=['NA', '?']) for muon_file in sorted(muon_files)] 
df_all_data = pd.concat(df_files, ignore_index=True)
df_good_data = df_all_data[(df_all_data['muon_efficiency'] < 1)] # & (df_all_data['ring_containment'] > 0.999) & (df_all_data['size_outside'] < 500)]# & (df_all_data['ring_completeness'] > 0.8)]
#df_good_data = df_all_data[(df_all_data['ring_containment'] > 0.999) & (df_all_data['size_outside'] < 500)]# & (df_all_data['ring_completeness'] > 0.8)]
del(df_all_data)
df_good_data


In [ ]:
upper_zenith_limit = 80
lower_zenith_limit = 60
upper_azimuth_limit = 185
lower_azimuth_limit = 175

# Choose only runs with zenith anlge < 5 degrees and then create two tables with the time of start and time of end
# runnumbers = cis['runnumber'][(cis['cos_zenith'] >= np.cos(np.deg2rad(upper_zenith_limit))) & 
#                               (cis['cos_zenith'] <= np.cos(np.deg2rad(lower_zenith_limit))) & 
#                               (cis['az_tel'] >= np.deg2rad(lower_azimuth_limit)) & 
#                               (cis['az_tel'] <= np.deg2rad(upper_azimuth_limit))]
runnumbers = runsummary['runnumber'][
    (runsummary['mean_altitude'] >= np.deg2rad(lower_zenith_limit)) & 
    (runsummary['mean_altitude'] <= np.deg2rad(upper_zenith_limit)) & 
    (runsummary['mean_azimuth'] >= np.deg2rad(lower_azimuth_limit)) & 
    (runsummary['mean_azimuth'] <= np.deg2rad(upper_azimuth_limit))
]
#zenith_table_cis = cis[cis['runnumber']].isin(runnumbers)
zenith_table = runsummary[runsummary['runnumber'].isin(runnumbers)]
start_table = zenith_table['time']
end_table = zenith_table['time']+zenith_table['elapsed_time']
# for index,row in new_table.iterrow:
#     print(row)
#for start, end in zip(start_table, end_table):
    #print(f"start = {start} / end = {end} and difference = {start - end}")

#zenith_table.to_csv('/Users/vdk/muons2024/data_zenith_cuts/zenith<5deg.csv', index=False)
zenith_table


In [ ]:
max(runsummary['mean_azimuth'])

### Choose muon events that satisfy the condition

In [ ]:
# Conditions: Select rows where Column1 is greater than 2 and Column2 is 'D' or 'E'
#df_zenith_cut = pd.read_csv('/Users/vdk/muons2024/data_zenith_cuts/zenith<5deg.csv', na_values=['NA', '?'])
df_zenith_cut = zenith_table
filtered_frames = []
for index, row in df_zenith_cut.iterrows():
    #print(f"time of start of the run = {datetime.fromtimestamp(row['time'][index]).strftime('%Y-%m-%d %H:%M:%S.%f')} and end of the run = {datetime.fromtimestamp(max(row['time'][index] + row['elapsed_time'][index])).strftime('%Y-%m-%d %H:%M:%S.%f')}")
    filtered_df = df_good_data[(df_good_data['event_time'] >= row['time']) & (df_good_data['event_time'] <= (row['time'] + row['elapsed_time']))]
    filtered_frames.append(filtered_df)
    #df_cut_data = pd.concat(filtered_df, ignore_index=True)

In [ ]:
final_df = pd.concat(filtered_frames, ignore_index=True)
#final_df = final_df[(final_df['ring_completeness'] > 0.5)]
final_df

In [ ]:
alpha_value = 1
first_column = np.array(first_column)
second_column = np.array(second_column)

fig, ax = plt.subplots(figsize=(10,10))
# Create a circle patch
#circle = Circle((0, 0), 11.5, color='red', fill=False, label = 'LST1 full mirror')  # Center at (0.5, 0.5) with radius 0.2

# Add the circle to the Axes
#ax.add_patch(circle)

# for mirror_x, mirror_y in zip(first_column,second_column):
#     circle = Circle((mirror_y*0.01, (mirror_x*0.01-0.269)), 0.75, color='k', fill=False, alpha = 0.5)  # Center at (0.5, 0.5) with radius 0.2
#     ax.add_patch(circle)
# Set the aspect of the plot to equal to make sure the circle is not oval

for center_x, center_y in zip(first_column, second_column):
    hex = RegularPolygon((center_y*0.01, center_x*0.01-0.269), numVertices=6, radius=0.85, alpha=0.4, edgecolor='k', fill=False)
    ax.add_patch(hex)
plt.autoscale(enable = True)

ax.set_aspect('equal')

ax.scatter(final_df['impact_x_array'][
    (final_df['event_time'] > start_date_2023) & 
    (final_df['event_time'] < end_date_2024)], 
           final_df['impact_y_array'][
    (final_df['event_time'] > start_date_2023) & 
    (final_df['event_time'] < end_date_2024)
], alpha = alpha_value, s = 1, marker = 'x')

ax.set_xlabel("Impact parameter X [m]") 
ax.set_ylabel("Impact parameter Y [m]")
ax.set_title("Reconstructed impact points of the observed muons on the LST mirror, zenith 10-20 deg")
handles, labels = ax.get_legend_handles_labels()
#ax.legend(handles, labels)
#plt.savefig(f'/Users/vdk/muons2024/images/ZenithImpactStudy/ImpactPoints_zenith20-30_2023year_2.png', dpi=200, format='png', bbox_inches='tight')
plt.grid(alpha=0.15)
plt.show()

In [ ]:
listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/0784-reflection/tables/*')
listdir2 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/0684-reflection/tables/*')
listdir3 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/0884-reflection/tables/*')


dat = Table.read('{}'.format(listdir[0]), format='fits')
for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')
    dat = vstack([dat, dat2])
df = dat.to_pandas()
df_sim_data = df[df['good_ring'] & (df['muon_efficiency'] < 1)]# & (df['ring_containment'] > 0.999) & (df['size_outside'] < 500) & (df['ring_completeness'] > 0.9)]


dat = Table.read('{}'.format(listdir2[0]), format='fits')
for muon_file in listdir2:
    dat2 = Table.read(muon_file, format='fits')
    dat = vstack([dat, dat2])
df_2 = dat.to_pandas()
df_sim_data_2 = df_2[df_2['good_ring'] & (df_2['muon_efficiency'] < 1)]# & (df['ring_containment'] > 0.999) & (df_2['size_outside'] < 500) & (df_2['ring_completeness'] > 0.9)]


# dat = Table.read('{}'.format(listdir3[0]), format='fits')
# for muon_file in listdir3:
#     dat2 = Table.read(muon_file, format='fits')
#     dat = vstack([dat, dat2])
# df_3 = dat.to_pandas()
# df_good_data_3 = df_3[df_3['good_ring'] & (df['ring_containment'] > 0.999) & (df['muon_efficiency'] < 1) & (df['size_outside'] < 500) & (df_3['ring_completeness'] > 0.9)]

df_sim_data_2

In [ ]:
size_outside = 500
ring_completeness = 0.9
ring_containment = 0.9
bin_number = 10

fig,ax = plt.subplots()
fig2,ax2 = plt.subplots()

sns.regplot(x = final_df['ring_radius'][(final_df['ring_completeness'] > ring_completeness) & 
                                        (final_df['size_outside'] < size_outside) &   
                                        (final_df['ring_containment'] > ring_containment)], 
            y = final_df['ring_width'][(final_df['ring_completeness'] > ring_completeness) & 
                                       (final_df['size_outside'] < size_outside) &  
                                       (final_df['ring_containment'] > ring_containment)], 
            x_bins=bin_number, ax=ax, color = 'k', label = 'real data')

sns.regplot(x = df_sim_data['ring_radius'][(df_sim_data['ring_completeness'] > ring_completeness) & 
                                           (df_sim_data['size_outside'] < size_outside) & 
                                           (df_sim_data['ring_containment'] > ring_containment)], 
            y = df_sim_data['ring_width'][(df_sim_data['ring_completeness'] > ring_completeness) & 
                                          (df_sim_data['size_outside'] < size_outside) & 
                                          (df_sim_data['ring_containment'] > ring_containment)], 
            x_bins=bin_number, ax=ax, color = 'r', label = '100% opt eff')

sns.regplot(x = df_sim_data_2['ring_radius'][(df_sim_data_2['ring_completeness'] > ring_completeness) & 
                                             (df_sim_data_2['size_outside'] < size_outside) &
                                             (df_sim_data_2['ring_containment'] > ring_containment)], 
            y = df_sim_data_2['ring_width'][(df_sim_data_2['ring_completeness'] > ring_completeness) & 
                                            (df_sim_data_2['size_outside'] < size_outside) & 
                                            (df_sim_data_2['ring_containment'] > ring_containment)], 
            x_bins=bin_number, ax=ax, color = 'green', label = '88% opt eff')

sns.regplot(x = final_df['ring_radius'][(final_df['ring_completeness'] > ring_completeness) & 
                                        (final_df['size_outside'] < size_outside) & 
                                        (final_df['ring_containment'] > ring_containment)], 
            y = final_df['ring_size'][(final_df['ring_completeness'] > ring_completeness) & 
                                      (final_df['size_outside'] < size_outside) & 
                                      (final_df['ring_containment'] > ring_containment)], 
            x_bins=bin_number, ax=ax2, color = 'k', label = 'real data')
sns.regplot(x = df_sim_data['ring_radius'][(df_sim_data['ring_completeness'] > ring_completeness) &
                                           (df_sim_data['size_outside'] < size_outside) & 
                                           (df_sim_data['ring_containment'] > ring_containment)], 
            y = df_sim_data['ring_size'][(df_sim_data['ring_completeness'] > ring_completeness) & 
                                         (df_sim_data['size_outside'] < size_outside) & 
                                         (df_sim_data['ring_containment'] > ring_containment)], 
            x_bins=bin_number, ax=ax2, color = 'r', label = '100% opt eff')
sns.regplot(x = df_sim_data_2['ring_radius'][(df_sim_data_2['ring_completeness'] > ring_completeness) & 
                                             (df_sim_data_2['size_outside'] < size_outside) &
                                             (df_sim_data_2['ring_containment'] > ring_containment)], 
            y = df_sim_data_2['ring_size'][(df_sim_data_2['ring_completeness'] > ring_completeness) & 
                                           (df_sim_data_2['size_outside'] < size_outside) &  
                                           (df_sim_data_2['ring_containment'] > ring_containment)], 
            x_bins=bin_number, ax=ax2, color = 'green', label = '88% opt eff')



format_axes(ax,'width')        
format_axes(ax2,'size') 

In [ ]:
usefull_func.hist_wo_outliers(df_sim_data_2['ring_size'], outlier_value=10000, mean_line=True, density=True)
usefull_func.hist_wo_outliers(df_sim_data['ring_size'], outlier_value=10000, mean_line=True, density=True)
usefull_func.hist_wo_outliers(final_df['ring_size'], outlier_value=10000, mean_line=True, density=True)

In [ ]:
len(df_sim_data)

In [ ]:
len(df_sim_data_2['ring_size'][(df_sim_data_2['ring_completeness'] > 0.9) & (df_sim_data_2['size_outside'] < size_outside) &  (df_sim_data_2['ring_size'] < 3000) & (df_sim_data_2['ring_containment'] > 0.999) & (df_sim_data_2['muon_efficiency'] < 1)])

In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'ring_size'

# Step 1: Create bins for 'ring_radius'
final_df[f'{x_value_str}_bin'] = pd.cut(final_df[x_value_str], bins=100)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = final_df.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Calculate number of elements in each bin
bin_counts = final_df.groupby(f'{x_value_str}_bin').size().reset_index(name='count')

# Merge the counts back with the mean calculations if needed
binned_data_with_counts = binned_data.merge(bin_counts, on=f'{x_value_str}_bin')


fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [4, 3]}, figsize=(12,12))
fig.subplots_adjust(hspace=0.05)
ax = axes[0]
ax.scatter(binned_data_with_counts['bin_mid'], binned_data_with_counts[y_value_str], label=f'Binned {y_value_str}', alpha = 0.8)
ax.set_ylabel('Size of the ring, [pe]')
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
ax.plot(final_df[x_value_str], m*final_df[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)
ax.legend()
bx = axes[1]
x_pos = list(binned_data_with_counts['bin_mid'])
y_pos = np.array(binned_data_with_counts['count'])
y_pos = y_pos / sum(y_pos)
bx.bar(x_pos, y_pos, width=0.01, label="Number of events")
#bx.set_ylim(0,25000)
bx.set_ylabel('Probability density')
ax.grid(alpha = 0.5)
bx.grid(alpha = 0.5)
bx.legend()

bx.tick_params(axis="y", labelsize=9)
# Scatter plot of binned values


plt.xlabel('Radius of the ring, [deg]')
plt.legend()
plt.grid(alpha=0.5)


### LST mirrors

In [ ]:
alpha_value = 0.2
first_column = np.array(first_column)
second_column = np.array(second_column)

fig, ax = plt.subplots(figsize=(10,10))
# Create a circle patch
#circle = Circle((0, 0), 11.5, color='red', fill=False, label = 'LST1 full mirror')  # Center at (0.5, 0.5) with radius 0.2

# Add the circle to the Axes
#ax.add_patch(circle)

# for mirror_x, mirror_y in zip(first_column,second_column):
#     circle = Circle((mirror_y*0.01, (mirror_x*0.01-0.269)), 0.75, color='k', fill=False, alpha = 0.5)  # Center at (0.5, 0.5) with radius 0.2
#     ax.add_patch(circle)
# Set the aspect of the plot to equal to make sure the circle is not oval

for center_x, center_y in zip(first_column, second_column):
    hex = RegularPolygon((center_y*0.01, center_x*0.01-0.269), numVertices=6, radius=0.85, alpha=0.4, edgecolor='k', fill=False)
    ax.add_patch(hex)
plt.autoscale(enable = True)

ax.set_aspect('equal')

ax.scatter(final_df['impact_x_array'], final_df['impact_y_array'], alpha = alpha_value, s = 1, marker = 'x')
ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
ax.set_title("Reconstructed impact points of the muons on the LST mirror, transparency of dots = 1")
handles, labels = ax.get_legend_handles_labels()
#ax.legend(handles, labels)
plt.savefig(f'/Users/vdk/muons2024/images/ZenithImpactStudy/ImpactPoints_zenith{lower_zenith_limit-upper_zenith_limit}.png', dpi=200, format='png', bbox_inches='tight')
plt.grid(alpha=0.15)
plt.show()

In [ ]:
final_df

In [ ]:
upper_zenith_limit = 40
lower_zenith_limit = 45
upper_azimuth_limit = 110
lower_azimuth_limit = 106

# Choose only runs with zenith anlge < 5 degrees and then create two tables with the time of start and time of end
runnumbers = cis['runnumber'][(cis['cos_zenith'] >= np.cos(np.deg2rad(upper_zenith_limit))) & 
                              (cis['cos_zenith'] <= np.cos(np.deg2rad(lower_zenith_limit))) & 
                              (cis['az_tel'] >= np.deg2rad(lower_azimuth_limit)) & 
                              (cis['az_tel'] <= np.deg2rad(upper_azimuth_limit))]
#zenith_table_cis = cis[cis['runnumber']].isin(runnumbers)
zenith_table = runsummary[runsummary['runnumber'].isin(runnumbers)]
start_table = zenith_table['time']
end_table = zenith_table['time']+zenith_table['elapsed_time']
# for index,row in new_table.iterrow:
#     print(row)
#for start, end in zip(start_table, end_table):
    #print(f"start = {start} / end = {end} and difference = {start - end}")

#zenith_table.to_csv('/Users/vdk/muons2024/data_zenith_cuts/zenith<5deg.csv', index=False)
zenith_table
# Conditions: Select rows where Column1 is greater than 2 and Column2 is 'D' or 'E'
filtered_frames = []
for index, row in zenith_table.iterrows():
    print(f"time of start of the run = {row['time']} and end of the run = {row['time'] + row['elapsed_time']}")
    filtered_df = df_good_data[(df_good_data['event_time'] >= row['time']) & (df_good_data['event_time'] <= (row['time'] + row['elapsed_time']))]
    filtered_frames.append(filtered_df)
    #df_cut_data = pd.concat(filtered_df, ignore_index=True)    

final_df = pd.concat(filtered_frames, ignore_index=True)
final_df



In [ ]:
# Create a new figure and an axes instance
fig, ax = plt.subplots(figsize=(12,12))
# Create a circle patch
circle = Circle((0, 0), 11.5, color='red', fill=False, label = 'LST1 full mirror')  # Center at (0.5, 0.5) with radius 0.2

alpha_value = 0.5

# Add the circle to the Axes
ax.add_patch(circle)
for center_x, center_y in zip(first_column, second_column):
    hex = RegularPolygon((center_y*0.01, center_x*0.01-0.269), numVertices=6, radius=0.85, alpha=0.7, edgecolor='k', fill=False)
    ax.add_patch(hex)
plt.autoscale(enable = True)

ax.set_aspect('equal')
# Set the aspect of the plot to equal to make sure the circle is not oval
ax.set_aspect('equal')

ax.scatter(final_df['impact_x_array'], final_df['impact_y_array'], alpha = alpha_value, s = 1, marker = 'x')
ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
ax.set_title("Reconstructed impact points of the muons on the LST mirror, transparency of dots = 1")
handles, labels = ax.get_legend_handles_labels()
#ax.legend(handles, labels)
#plt.savefig('/Users/vdk/muons2024/images/ImpactPoints_19-23Y_Alpha1.png', dpi=200, format='png', bbox_inches='tight')
plt.grid(alpha=0.15)
plt.show()

In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'ring_size'

# Step 1: Create bins for 'ring_radius'
final_df[f'{x_value_str}_bin'] = pd.cut(final_df[x_value_str], bins=100)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = final_df.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Calculate number of elements in each bin
bin_counts = final_df.groupby(f'{x_value_str}_bin').size().reset_index(name='count')

# Merge the counts back with the mean calculations if needed
binned_data_with_counts = binned_data.merge(bin_counts, on=f'{x_value_str}_bin')


fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [4, 3]}, figsize=(12,12))
fig.subplots_adjust(hspace=0.05)
ax = axes[0]
ax.scatter(binned_data_with_counts['bin_mid'], binned_data_with_counts[y_value_str], label=f'Binned {y_value_str}', alpha = 0.8)
ax.set_ylabel('Size of the ring, [p.e.]')
m, b = np.polyfit(final_df[x_value_str], final_df[y_value_str], 1)
ax.plot(final_df[x_value_str], m*final_df[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)
ax.legend()
bx = axes[1]
x_pos = list(binned_data_with_counts['bin_mid'])
y_pos = np.array(binned_data_with_counts['count'])
y_pos = y_pos / sum(y_pos)
bx.bar(x_pos, y_pos, width=(max(x_pos)-min(x_pos))/(len(x_pos)-2), label="Number of events", alpha = 0.7)
#bx.set_ylim(0,25000)
bx.set_ylabel('Probability density')
ax.grid(alpha = 0.5)
bx.grid(alpha = 0.5)
bx.legend()

bx.tick_params(axis="y", labelsize=9)
# Scatter plot of binned values


plt.xlabel('Radius of the ring, [deg]')
plt.legend()
plt.grid(alpha=0.5)


In [ ]:
2+2